In [1]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg

import pandas as pd


##Load Dataset
data=gutenberg.raw('shakespeare-hamlet.txt')

## Save to the file
with open('hamlet.txt','w') as f:
  f.write(data)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
## Data Preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

##Load the dataset
with open('hamlet.txt','r') as f:
  text=f.read().lower()

## Tokenization
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
total_words

4818

In [3]:
input_sequences=[]
for line in text.split('\n'):
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [4]:
input_sequences[:10]

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891]]

In [5]:
##pad Sequences
max_sequence_len=max([len(x) for x in input_sequences])
input_sequences=np.array(pad_sequences(input_sequences,maxlen=max_sequence_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [6]:
## Create predictors and lable
import tensorflow as tf
x,y=input_sequences[:,:-1],input_sequences[:,-1]
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [7]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [51]:
## Train our LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.optimizers import Adam

##Define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

## Compile
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [52]:
## Train with GRU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,GRU
from tensorflow.keras.optimizers import Adam

##Define the model
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(GRU(150,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(100))
model.add(Dense(total_words,activation='softmax'))

## Compile
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop=EarlyStopping(monitor='val_loss',patience=10,verbose=1)

In [16]:
history =model.fit(x_train,y_train,epochs=100,validation_data=(x_test,y_test),verbose=1)


Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 45s 70ms/step - accuracy: 0.3270 - loss: 3.1801 - val_accuracy: 0.0536 - val_loss: 9.7926
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 70ms/step - accuracy: 0.3306 - loss: 3.1143 - val_accuracy: 0.0550 - val_loss: 9.9165
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 81s 69ms/step - accuracy: 0.3448 - loss: 3.0882 - val_accuracy: 0.0565 - val_loss: 9.9919
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 45s 69ms/step - accuracy: 0.3627 - loss: 3.0077 - val_accuracy: 0.0560 - val_loss: 10.1100
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 83s 71ms/step - accuracy: 0.3615 - loss: 2.9684 - val_accuracy: 0.0560 - val_loss: 10.1980
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 80s 69ms/step - accuracy: 0.3789 - loss: 2.9118 - val_accuracy: 0.0571 - val_loss: 10.2978
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 82s 68ms/step - accuracy: 0.3759 - loss: 2.8911 - val_accuracy: 0.0544 - val_loss: 10.3771
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 44s 69ms/step - accuracy: 0.3867 - los

In [26]:
def predict_next_word(seed_text, model, tokenizer, max_sequence_len):
    # Convert the seed text to a sequence of tokens
    token_list = tokenizer.texts_to_sequences([seed_text])[0]

    # Truncate or pad the token list as needed
    if len(token_list) >= max_sequence_len:
        token_list = token_list[-(max_sequence_len-1):]

    # Pad the sequence to the required length
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

    # Predict the next word
    predicted = model.predict(token_list, verbose=0)

    # Get the index of the predicted word
    predicted_word_index = np.argmax(predicted, axis=-1)[0]  # Ensure to access the first element

    # Find the word corresponding to the predicted index
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None


In [48]:
input_text='Now is the winter of our'
print(f'Input Text: {input_text}')
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(input_text,model,tokenizer,max_sequence_len)
print(f'Next Word: {next_word}')

Input Text: Now is the winter of our
Next Word: mother


In [49]:
model.save('my_model.keras')
import pickle
## save the tokenizer
with open ('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
## if you want to run through google colab try this

!pip install streamlit ngrok
!pip install streamlit pyngrok
!ngrok authtoken <your_authtoken>

# Install streamlit and pyngrok
!pip install streamlit pyngrok

# Import necessary libraries
import streamlit as st
from pyngrok import ngrok

# Import necessary libraries
from pyngrok import ngrok
import streamlit as st

# Create the Streamlit app
st.title("Customer Churn Prediction")
# (Add the rest of your app logic here)

# Create a tunnel to the streamlit app running on port 8501
public_url = ngrok.connect(8501, "http")
print(f"Streamlit app URL: {public_url}")

# Run the Streamlit app (app.py should be the filename)
!streamlit run app.py &>/dev/null&
